In [1]:
import numpy as np
import pandas as pd
from utils import  amplitude_scaling, load_dataset, save_distance_matrix
from sklearn.neighbors import KNeighborsClassifier
from dtaidistance import dtw
from sklearn.metrics import classification_report 
from sklearn.model_selection import GridSearchCV, cross_val_predict
from utils import dtw_dist, decode_y, top_flop_f1
from pipeline import Pipeline
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    classification_report,
    confusion_matrix,
    roc_auc_score,
    make_scorer
)
from scikitplot.metrics import plot_roc
from scikitplot.metrics import plot_precision_recall
import matplotlib.pyplot as plt

# TS APPROX

In [2]:
X_train_paa, y_train_paa, X_test_paa, y_test_paa, enc_dict = load_dataset('../dataset_ts/ts_data_paa128.npz')

In [3]:
X_train_paa_scaled = np.squeeze(X_train_paa)
X_test_paa_scaled = np.squeeze(X_test_paa)
y_train_paa_scaled = y_train_paa
y_test_paa_scaled = y_test_paa
X_train_paa_scaled = np.expand_dims(X_train_paa_scaled, axis=1)
X_test_paa_scaled = np.expand_dims(X_test_paa_scaled, axis=1)

In [5]:
"""
#PER AMPLITUDE SCALING, non utilizzato
X_train_paa_scaled, y_train_paa_scaled = amplitude_scaling(X_train_paa, y_train_paa)
X_test_paa_scaled, y_test_paa_scaled = amplitude_scaling(X_test_paa, y_test_paa)

X_train_paa_scaled = np.expand_dims(X_train_paa_scaled, axis=1)
X_test_paa_scaled = np.expand_dims(X_test_paa_scaled, axis=1)
"""

In [4]:
X_train_paa_scaled.shape, np.mean(X_train_paa_scaled[0,:,:])

((7500, 1, 128), 2283.4902478449)

In [5]:
from sktime.classification.kernel_based import RocketClassifier

### Test Results

In [6]:
rocket = RocketClassifier(n_jobs=-1, rocket_transform='rocket')
rocket.fit(X_train_paa, y_train_paa)
y_pred = rocket.predict(X_test_paa)
print(classification_report(decode_y(y_test_paa, enc_dict), decode_y(y_pred, enc_dict)))

                   precision    recall  f1-score   support

              emo       0.18      0.17      0.17       125
             folk       0.07      0.06      0.07       125
             goth       0.06      0.06      0.06       125
            happy       0.46      0.58      0.51       125
      heavy-metal       0.11      0.12      0.12       125
       honky-tonk       0.23      0.23      0.23       125
           j-idol       0.17      0.18      0.17       125
             kids       0.14      0.15      0.14       125
   minimal-techno       0.58      0.61      0.59       125
              mpb       0.07      0.06      0.07       125
          new-age       0.20      0.25      0.22       125
            opera       0.23      0.26      0.24       125
            piano       0.15      0.14      0.14       125
progressive-house       0.33      0.34      0.34       125
            salsa       0.20      0.18      0.19       125
        sertanejo       0.15      0.13      0.14       

### TS RAW

In [7]:
X_train, y_train, X_test, y_test, enc_dict = load_dataset('../dataset_ts/ts_data_raw.npz')

In [8]:
X_train_scaled = np.squeeze(X_train)
X_test_scaled = np.squeeze(X_test)
y_train_scaled = y_train
y_test_scaled = y_test
X_train_scaled = np.expand_dims(X_train_scaled, axis=1)
X_test_scaled = np.expand_dims(X_test_scaled, axis=1)

In [16]:
"""
#PER AMPLITUDE SCALING, non utilizzato
X_train_scaled, y_train_scaled = amplitude_scaling(X_train, y_train)
X_test_scaled, y_test_scaled = amplitude_scaling(X_test, y_test)

X_train_scaled = np.expand_dims(X_train_scaled, axis=1)
X_test_scaled = np.expand_dims(X_test_scaled, axis=1)
"""

### Test Results

In [9]:
rocket = RocketClassifier(n_jobs=-1, rocket_transform='rocket')
rocket.fit(X_train_scaled, y_train_scaled)
y_pred = rocket.predict(X_test_scaled)
print(classification_report(decode_y(y_test_scaled, enc_dict), decode_y(y_pred, enc_dict)))

                   precision    recall  f1-score   support

              emo       0.35      0.32      0.34       125
             folk       0.15      0.12      0.13       125
             goth       0.15      0.09      0.11       125
            happy       0.71      0.78      0.75       125
      heavy-metal       0.35      0.41      0.38       125
       honky-tonk       0.49      0.54      0.51       125
           j-idol       0.38      0.34      0.36       125
             kids       0.25      0.27      0.26       125
   minimal-techno       0.68      0.72      0.70       125
              mpb       0.15      0.13      0.14       125
          new-age       0.37      0.42      0.39       125
            opera       0.44      0.50      0.47       125
            piano       0.42      0.44      0.43       125
progressive-house       0.52      0.52      0.52       125
            salsa       0.57      0.67      0.62       125
        sertanejo       0.28      0.30      0.29       